# PageRank Implementation on Twitter Congress Dataset
This notebook implements the PageRank algorithm from scratch under both unweighted and weighted edge settings.

## 📄 Code Documentation (Simple & Humanized)

### 🔸 What This Code Does
This code implements the **PageRank algorithm** from scratch. It calculates how “important” each person (node) is in a Twitter network of U.S. Congress members based on how they are connected.

We use two versions of PageRank:
1. **Unweighted PageRank**: All connections are treated equally.
2. **Weighted PageRank**: Some connections are stronger than others (e.g., more retweets or replies), so they pass more influence.

---

### 🔸 How the Graph Is Represented
We represent the Twitter network as a **directed graph**, which means each connection goes from one person to another.

We use two dictionaries:
- `adjacency`: tells us who each person is connected to (i.e., who they follow or interact with).
- `weights`: tells us how strong each connection is. For example, if user A has 3 connections and one is stronger than the others, that one gets more influence in weighted PageRank.

Both dictionaries are built using a function called `load_graph()`, which reads the file and stores the data in a way that’s easy to use later.

---

### 🔸 What Is PageRank Doing?
Think of each node (person) starting with the same amount of "influence." At every step, they give away their influence to others they are connected to.

- In **unweighted**, it’s shared equally among all connections.
- In **weighted**, it’s shared based on how strong each connection is.

We repeat this process multiple times until the scores stop changing much — that’s when we say the algorithm has **converged**.

---

### 🔸 Stopping Condition (When to Stop Iterating)
We stop when one of these happens:
- The scores don’t change much anymore (difference is less than `1e-6`).
- We hit the maximum number of steps (default is 100 iterations).

This helps avoid running forever and ensures the result is stable.

---

### 🔸 What the Output Shows
At the end, we display the **top 25 people with the highest PageRank score** for both unweighted and weighted versions.

This tells us who is the most influential in the network based on connections — either treating all connections the same, or taking connection strength into account.

In [ ]:
from collections import defaultdict
import pandas as pd

def load_graph(file_path):
    adjacency = defaultdict(list)
    weights = defaultdict(lambda: defaultdict(float))

    with open(file_path, 'r') as f:
        for line in f:
            src, dst = line.strip().split()
            adjacency[src].append(dst)
            weights[src][dst] += 1.0

    return adjacency, weights

def pagerank_unweighted(graph, damping=0.85, max_iter=100, tol=1e-6):
    nodes = set(graph.keys())
    for targets in graph.values():
        nodes.update(targets)
    nodes = list(nodes)
    N = len(nodes)
    pr = {node: 1.0 / N for node in nodes}

    for _ in range(max_iter):
        new_pr = {node: (1 - damping) / N for node in nodes}
        for node in nodes:
            out_links = graph.get(node, [])
            if not out_links:
                continue
            share = pr[node] / len(out_links)
            for neighbor in out_links:
                new_pr[neighbor] += damping * share
        if all(abs(new_pr[n] - pr[n]) < tol for n in nodes):
            break
        pr = new_pr
    return pr

def pagerank_weighted(graph, weights, damping=0.85, max_iter=100, tol=1e-6):
    nodes = set(graph.keys())
    for targets in graph.values():
        nodes.update(targets)
    nodes = list(nodes)
    N = len(nodes)
    pr = {node: 1.0 / N for node in nodes}

    for _ in range(max_iter):
        new_pr = {node: (1 - damping) / N for node in nodes}
        for node in nodes:
            neighbors = graph.get(node, [])
            total_weight = sum(weights[node].values())
            if total_weight == 0:
                continue
            for neighbor in neighbors:
                weight = weights[node][neighbor]
                new_pr[neighbor] += damping * pr[node] * (weight / total_weight)
        if all(abs(new_pr[n] - pr[n]) < tol for n in nodes):
            break
        pr = new_pr
    return pr

def top_k(pr_dict, k=25):
    return sorted(pr_dict.items(), key=lambda x: x[1], reverse=True)[:k]

In [ ]:
# Load the dataset
adjacency, weights = load_graph('congress.edges')

In [ ]:
# Run unweighted PageRank
unweighted_pr = pagerank_unweighted(adjacency)
top_25_unweighted = top_k(unweighted_pr)
pd.DataFrame(top_25_unweighted, columns=["Node", "Unweighted PR"])

In [ ]:
# Run weighted PageRank
weighted_pr = pagerank_weighted(adjacency, weights)
top_25_weighted = top_k(weighted_pr)
pd.DataFrame(top_25_weighted, columns=["Node", "Weighted PR"])

In [ ]:
# Comparison of Top 25
df_unweighted = pd.DataFrame(top_25_unweighted, columns=["Node", "Unweighted PR"])
df_weighted = pd.DataFrame(top_25_weighted, columns=["Node", "Weighted PR"])
comparison = pd.merge(df_unweighted, df_weighted, on="Node", how="outer")
comparison